# Optimal investment and withdrawal strategies for Education 529 Plans
## Abstract

A research study focuses on the optimal investment and withdrawal strategies for Education 529 Plans to fund college education expenses over 18 years. The study aims to investigate the impact of different investment frequencies (monthly, quarterly, semiannually, and annually) on the growth and effectiveness of the 529 plan, followed by various withdrawal frequencies (monthly, semiannual, and annual) to cover four years of college expenses.

**Objective**: The primary objective of this research is to identify the most effective investment and withdrawal strategies within the context of Education 529 Plans. Specifically, we aim to determine the optimal investment frequency and withdrawal frequency that would maximize the plan's growth and ensure sufficient funds for covering college bills.

**Research Questions**:


+  How does the investment frequency (monthly, quarterly, semiannually, and annually) affect the growth and overall value of the Education 529 Plan over 18 years?
+ What are the implications of different withdrawal frequencies (monthly, semiannual, and annual) on the available funds to cover four years of college expenses?
+  Which combination of investment and withdrawal frequencies would provide the best outcomes in maximizing the plan's growth and meeting college funding requirements?


**Methodology**:

+ Data Collection: Gather historical investment performance and college expenses over 18 years. Collect data on different investment and withdrawal frequencies from existing Education 529 Plans.
+  Simulation: Utilize a comprehensive financial modeling approach to simulate the growth of the Education 529 Plan under various investment frequencies. Incorporate different withdrawal frequencies to assess the impact on available funds for college expenses.

+  Analyze the simulated data to identify patterns, trends, and key findings regarding the effectiveness of different investment and withdrawal strategies.

Recommendation: Based on the analysis, provide guidance for the best investment and withdrawal practices for Education 529


**Python Coding**


+ Based on the Robert Stock market data (http://www.econ.yale.edu/~shiller/data.htm), we focus on the SP500 (large cap) and 10-year Bond's monthly return.

+ Rolling window Back testing on historical data

 + Loop over different percentage of equities, SP500= 0, 0.05, 0.10, 0.15, ...,1.0

     + loop over different investment strategies (monthly, quarterly, semiannually, annually )
        + loop over the rolling window and update the investments
+ Monte Carlo Simulation (https://www.investopedia.com/terms/m/montecarlosimulation.asp)

Reference
+  Lump-sum investing versus cost averaging: Which is better? (https://investor.vanguard.com/investor-resources-education/news/lump-sum-investing-versus-cost-averaging-which-is-better)
+ Cost averaging: Invest now or temporarily hold your cash?https://corporate.vanguard.com/content/dam/corp/research/pdf/cost_averaging_invest_now_or_temporarily_hold_your_cash.pdf
+ Seattle undergraduate student budget: https://admit.washington.edu/costs/coa/
+ UW Tuition History: https://depts.washington.edu/opbfiles/web/2016-17%20Tuition%20&%20Fee%20History.pdf


## Load and Clean the Data

 Robert Stock market data (http://www.econ.yale.edu/~shiller/data.htm), we focus on the SP500 (large cap) and 10-year Bond's monthly return

In [ ]:
import time
start_time = time.time()

import  pandas as pd
returns_df = pd.read_excel('ie_data.xls', sheet_name='Data', header=7)
returns_df

,Date,P,D,E,Monthly_Total_SP500_Return,CPI,Fraction,Rate GS10,Price,Dividend,...,Unnamed: 14,TR CAPE,Unnamed: 16,Yield,Returns,Monthly_Total_Bond_Return,Returns.1,Real Return,Real Return.1,Returns.2
0,1871.01,4.44,0.26,0.4,NaN,12.464061,1871.041667,5.32,108.541508,6.356034,...,NaN,NaN,NaN,NaN,1.004177,NaN,1.000000,0.130609,0.092504,0.038106
1,1871.02,4.5,0.26,0.4,0.018393,12.844641,1871.125000,5.323333,106.748796,6.167708,...,NaN,NaN,NaN,NaN,1.004180,0.000003,0.974424,0.130858,0.094635,0.036224
2,1871.03,4.61,0.26,0.4,0.029259,13.034972,1871.208333,5.326667,107.761414,6.077650,...,NaN,NaN,NaN,NaN,1.004183,0.000003,0.964209,0.130951,0.096186,0.034765
3,1871.04,4.74,0.26,0.4,0.032899,12.559226,1871.291667,5.33,114.997369,6.307873,...,NaN,NaN,NaN,NaN,1.004185,0.000003,1.004919,0.122056,0.090972,0.031084
4,1871.05,4.86,0.26,0.4,0.029887,12.273812,1871.375000,5.333333,120.650541,6.454556,...,NaN,NaN,NaN,NaN,1.004188,0.000003,1.032591,0.122638,0.089488,0.033150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,2023.04,4121.467368,NaN,NaN,NaN,303.363,2023.291667,3.46,4139.631752,NaN,...,NaN,31.102513,NaN,0.027112,0.993708,-0.025603,41.322519,NaN,NaN,NaN
1828,2023.05,4146.173182,NaN,NaN,NaN,304.127,2023.375000,3.57,4153.984909,NaN,...,NaN,31.138963,NaN,0.026023,0.988087,-0.005656,40.959373,NaN,NaN,NaN
1829,2023.06,4345.372857,NaN,NaN,NaN,304.509,2023.458333,3.75,4348.098446,NaN,...,NaN,32.520839,NaN,0.022609,0.998176,0.010211,40.420668,NaN,NaN,NaN
1830,2023.07,4455.59,NaN,NaN,NaN,304.7,2023.541667,3.81,4455.590000,NaN,...,NaN,33.251184,NaN,0.021281,NaN,NaN,40.321663,NaN,NaN,NaN


In [ ]:
#select only three columns that related to this project
returns_df = returns_df.loc[1:, ['Date','Monthly_Total_SP500_Return', 'Monthly_Total_Bond_Return']]
print(f" There are {len(returns_df.index) } month's data!")

print(f" The missing values in each column ")
print(returns_df.isnull().sum())
returns_df.tail(20)

 There are 1831 month's data!
 The missing values in each column 
Date                          1
Monthly_Total_SP500_Return    4
Monthly_Total_Bond_Return     2
dtype: int64


,Date,Monthly_Total_SP500_Return,Monthly_Total_Bond_Return
1812,2022.01,-0.020510,0.011571
1813,2022.02,-0.029016,-0.002424
1814,2022.03,-0.008916,-0.036318
1815,2022.04,0.001196,0.043630
1816,2022.05,-0.078714,-0.007469
1817,2022.06,-0.033680,0.042048
1818,2022.07,0.004656,-0.020387
1819,2022.08,0.064483,-0.051711
1820,2022.09,-0.072765,0.015477
1821,2022.10,-0.030900,0.047051


In [ ]:
#Remove the missing values
returns_df = returns_df.dropna()

# reset the index due to the droping
returns_df.reset_index(inplace=True)
print(f" There are {len(returns_df.index) } month's data!")
print(f" The missing values in each column ")
print(returns_df.isnull().sum())
returns_df.tail(20)

 There are 1826 month's data!
 The missing values in each column 
index                         0
Date                          0
Monthly_Total_SP500_Return    0
Monthly_Total_Bond_Return     0
dtype: int64


,index,Date,Monthly_Total_SP500_Return,Monthly_Total_Bond_Return
1806,1807,2021.08,0.021860,-0.012039
1807,1808,2021.09,-0.000836,-0.010927
1808,1809,2021.10,0.004529,0.021668
1809,1810,2021.11,0.047455,0.006427
1810,1811,2021.12,0.002661,-0.034416
1811,1812,2022.01,-0.020510,0.011571
1812,1813,2022.02,-0.029016,-0.002424
1813,1814,2022.03,-0.008916,-0.036318
1814,1815,2022.04,0.001196,0.043630
1815,1816,2022.05,-0.078714,-0.007469


### Define the Cost of Attendance Constant

In [ ]:
# Current Non-resident total costs for attending UW Seattle, WA
Annual_total_costs = 63906
# Total costs inflation
Annual_total_costs_inflation = 0.06

In [ ]:
print(f" (Annual_total_costs, Annual_total_costs_inflation) = {(Annual_total_costs, Annual_total_costs_inflation)}")

 (Annual_total_costs, Annual_total_costs_inflation) = (63906, 0.06)


In [ ]:
# 1: monthly; 2: bimonthly; 3: quarterly; 6: semi-annual; 12: annually
Withdrawal_frequency_months =[ 1, 2, 3, 6, 12]
# withdrawal length in months for 4 years
Withdrawal_length_months = 4*12

### Define the Monthly Investment Constant

Historical Returns on Stocks, Bonds and Bills: 1928-2022

https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html

+ For 70% of SP500 and 30% of Bond, the Geometric Average Historical Return is about 9%.
+ The Annual total cost inflation of attendance is about 6%.
+ The **real return** for the investment is about 9%-6% =3%


In [ ]:
# 1: monthly; 2: bimonthly; 3: quarterly; 6: semi-annual; 12: annually
Investment_frequency_months =[ 1, 2, 3, 6, 12]
# investment length in months for 18 years
Investment_length_months = 18*12

Geometric_Average_Annual_Return = 0.09
real_annual_return = Geometric_Average_Annual_Return-Annual_total_costs_inflation
print(f" the real return after cost of attendance inflation = {real_annual_return:.2f}")

 the real return after cost of attendance inflation = 0.03


Convert annual rate of return to monthly rate of return, using the following formula

$$
(1+m)^{12}= (1+i)
$$
where m denotes the monthly return and r denotes the annual return

$$
m=(1+i)^{1/2}-1
$$

In [ ]:
month_return = pow(1+Geometric_Average_Annual_Return, 1/12)-1
print(f" the monthly return = {month_return:.4f}")

 the monthly return = 0.0072


### Estimate the Monthly Investment Amount

https://analystprep.com/study-notes/actuarial-exams/soa/fm-financial-mathematics/annuities-cash-flows-with-non-contingent-payments/

Suppose we need four times the total cost of attendance in 18 years since the students need to complete the college in 4 years;
by the formula


$$
\text{Total money need in 18 years}=4* \text{cost of attendance at year 18}
$$

Discount annuity factor
$$
\text{Discount annuity factor}=\dfrac{1-(1+r)^{-12*18}}{r}
$$

where $r$ is the real monthly return and 12*18 denotes the total number of months for 18 years

Then monthly investment

$$
\text{Then monthly investment} = \dfrac{\text{Total money need in 18 years}}{\text{Discount annuity factor}}
$$

$$
\text{Then annually investment} = \text{Then monthly investment} *12
$$

In [ ]:
total_money_needed = 4*Annual_total_costs
discount_annuity_factor = (1-pow(1+month_return, -Investment_length_months))/month_return
print(f"discount_annuity_factor={discount_annuity_factor:.2f}")
annually_investment_amount = total_money_needed/discount_annuity_factor*12
print(f"estimate annually_investment_amount ={annually_investment_amount:.2f}")

discount_annuity_factor=109.33
estimate annually_investment_amount =28056.09


## Execute the loop

In [ ]:
#generate stock percent=0.0.05, 0.1, 0.15, 0.2, ..., 0.9, 0.95, 1.0]
Stock_percent = [idx/100 for idx in range(0, 105,5)]

# find the last month row number for the rolling window
last_month_idx= len(returns_df.index)- Investment_length_months - Withdrawal_length_months
print(f" the total months of data = {len(returns_df.index)}")
print(f" the Investment_length_months = {Investment_length_months}")
print(f" the Withdrawal_length_months = {Withdrawal_length_months}")
print(f" the last month index for the starting index of the rolling window = {last_month_idx}")

 the total months of data = 1826
 the Investment_length_months = 216
 the Withdrawal_length_months = 48
 the last month index for the starting index of the rolling window = 1562


In [ ]:
len(Withdrawal_frequency_months)

5

## Results in N-dimensional Array
Muti-dimension array holds all the data; then we can run all the loops
+ First dimension holds the percentate;0, 0.05, 0.1, 0.15, ..., 1.0; which has 21 cases
+ Second dimension holds the rollowing windows starting location, which starts from 0, then 1, then 2, ..., last index
   for example, if it starts from 0, 1, 2, 3, 4, 5 respectively; which has 5+1 = 6 possibilites
+ Third dimension holds the investment (18*12=216) and withdrawal month (48) history
+ Fourth dimension holds the investmetn strategies (1, 2, 3, 6, 12 months); 5 possibilities
+ Fifth dimension holds the withdrawal strategies (1, 2, 3, 6, 12 months); 5 possibilities


In [ ]:
dim= (len(Stock_percent), last_month_idx+1, Investment_length_months + Withdrawal_length_months, len(Investment_frequency_months), len(Withdrawal_frequency_months))

dim

(21, 1563, 264, 5, 5)

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

In [ ]:
%%time
import numpy as np
dim= (len(Stock_percent), last_month_idx + 1, Investment_length_months + Withdrawal_length_months, len(Investment_frequency_months), len(Withdrawal_frequency_months))

# we need to have two investment accounts for sp500=stock and bond
stock_amount = np.zeros(dim)
bond_amount = np.zeros(dim)


# loop over the investment strategies for sp500; 0, 0.05, 0.1
for stock_pct_idx,stock_percent in enumerate(Stock_percent):
    print(f"stock_percent ={stock_percent}")
    # loop over the investment frequency
    for invest_period_idx, invest_period in enumerate(Investment_frequency_months):

        # loop over the start position for the rolling window
        for month_idx_start in range(0,last_month_idx+1):

            #loop over the investment length from relative index 0, 1, 2, ..., Investment_length_months -1
            for invest_relative_idx in range(0,Investment_length_months):

                # update the sp500  growth due to the past investment
                # a*=b means a =a*b
                #print(f" (stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx) ={(stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx)}")
                #print(f"returns_df.loc[month_idx_start + invest_relative_idx,'Monthly_Total_SP500_Return']= {returns_df.loc[month_idx_start + invest_relative_idx,'Monthly_Total_SP500_Return']}")
                # update the current balance = previous period's balance *(1+return) since investment is made at the end of period,
                if (invest_relative_idx !=0): # r_t= r_{t-1)*(1+r)
                    stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] = stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx -1, invest_period_idx, :] * \
                        (1+returns_df.loc[month_idx_start + invest_relative_idx,'Monthly_Total_SP500_Return'])
                    #print(f"stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :]={stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :]}")
                    # similily the bond growth due to the past investment
                    bond_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] = bond_amount[stock_pct_idx, month_idx_start,invest_relative_idx - 1, invest_period_idx, :] * \
                        (1+returns_df.loc[month_idx_start + invest_relative_idx,'Monthly_Total_Bond_Return'])
                else:# first period, there is no previous investment , which is 0
                    stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] = 0.0
                    bond_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] = 0.0
                # compute the new investment amount per period
                invest_amount_period= annually_investment_amount/12.0*invest_period

                # the sp500 and bond percentage is bond the specified percentage in the outer loop
                # update the sp500 and bond amount due to the new investment
                # invest_relative_idx = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                # add 1, it becomes idx =1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13
                # % rmainder
                if ((invest_relative_idx + 1) % invest_period == 0):
                    # invest in sp500 and bond according to the percentage of sp500 and bond respectively
                    stock_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] += invest_amount_period*stock_percent
                    bond_amount[stock_pct_idx, month_idx_start,invest_relative_idx, invest_period_idx, :] += invest_amount_period*(1-stock_percent)



stock_percent =0.0
stock_percent =0.05
stock_percent =0.1
stock_percent =0.15
stock_percent =0.2
stock_percent =0.25
stock_percent =0.3
stock_percent =0.35
stock_percent =0.4
stock_percent =0.45
stock_percent =0.5
stock_percent =0.55
stock_percent =0.6
stock_percent =0.65
stock_percent =0.7
stock_percent =0.75
stock_percent =0.8
stock_percent =0.85
stock_percent =0.9
stock_percent =0.95
stock_percent =1.0
Wall time: 12min 23s


In [ ]:
stock_amount.shape,bond_amount.shape

((21, 1563, 264, 5, 5), (21, 1563, 264, 5, 5))

## Balance the Portfolio by rolling SP500 into Bonds to Protect the Assets after the Last Investment


In [ ]:
portfolio = bond_amount + stock_amount

## Handle Withdraw

dim= (len(Stock_percent), last_month_idx+1, Investment_length_months + Withdrawal_length_months, len(Investment_frequency_months), len(Withdrawal_frequency_months))

In [ ]:
%%time
# loop over the investment strategies for sp500; 0, 0.05, 0.1
for stock_pct_idx,stock_percent in enumerate(Stock_percent):
    print(f"stock_percent ={stock_percent}")
    # loop over the investment frequency
    for invest_period_idx, invest_period in enumerate(Investment_frequency_months):

        #loop over withdrawal strategies
        for withdrawal_period_idx, withdrawal_period in enumerate(Withdrawal_frequency_months):
            # loop over Rolling window starting position for the rolling withdraw
            for month_idx_start in range(0,last_month_idx+1):
                #print(f"month_idx_start ={month_idx_start}")
                #loop over the windows relative index from 0, 1, ...WLM-1
                for withdrawal_relative_idx in range(0,Withdrawal_length_months):
                    #print(f"withdrawal_relative_idx={withdrawal_relative_idx}")
                    portfolio[stock_pct_idx, month_idx_start,Investment_length_months + withdrawal_relative_idx, invest_period_idx, withdrawal_period_idx] =  \
                        portfolio[stock_pct_idx, month_idx_start,Investment_length_months + withdrawal_relative_idx - 1, invest_period_idx, withdrawal_period_idx] * \
                            (1+returns_df.loc[month_idx_start + Investment_length_months + withdrawal_relative_idx,'Monthly_Total_Bond_Return'])
                   # computethe number of years for the withdraw
                    # // get the integer part, for example 12//10 = 1; 19//10 =1
                    # withdrawal_period_idx =0, 1,2, .., 11, 12, 13,
                    # think about //11?
                    num_year_withdrawal = (withdrawal_period_idx)//12
                    # compute the withdrawal amount per period
                    # new withdrawal amount = (1+inflation)^(number of years for investment+ number years withdrawal)
                    annually_withdrawal_amount = Annual_total_costs* pow(1+Annual_total_costs_inflation, Investment_length_months//12+num_year_withdrawal)
                    # consider the withdrawal strategies such as monthly 9withdrawal_period=1); bimonthly = (withdrawal_period=2)

                    withdrawal_amount_period= annually_withdrawal_amount/12.0*withdrawal_period

                    # deduct money from the bond account if the withdrawal_relative_idx match the withdrawal frequency/strategies
                    # a-=b; a =a-b
                    if ((withdrawal_relative_idx + 1) % withdrawal_period == 0):
                        portfolio[stock_pct_idx, month_idx_start,Investment_length_months + withdrawal_relative_idx, invest_period_idx, withdrawal_period_idx] -= withdrawal_amount_period


stock_percent =0.0
stock_percent =0.05
stock_percent =0.1
stock_percent =0.15
stock_percent =0.2
stock_percent =0.25
stock_percent =0.3
stock_percent =0.35
stock_percent =0.4
stock_percent =0.45
stock_percent =0.5
stock_percent =0.55
stock_percent =0.6
stock_percent =0.65
stock_percent =0.7
stock_percent =0.75
stock_percent =0.8
stock_percent =0.85
stock_percent =0.9
stock_percent =0.95
stock_percent =1.0
Wall time: 5min 14s


In [ ]:
portfolio.shape

(21, 1563, 264, 5, 5)

## Look at the Investment account balance before withdraw at T=12*18

dim= (len(Stock_percent), last_month_idx+1, Investment_length_months + Withdrawal_length_months, len(Investment_frequency_months), len(Withdrawal_frequency_months))
(# of port,# of rolling windows, investment and withdrawal history, investment strategies, withdraw strategies)



In [ ]:
investment_balance =portfolio
investment_balance = portfolio[:, :, -(Withdrawal_length_months+1), :, :]

In [ ]:
investment_balance.shape

(21, 1563, 5, 5)

In [ ]:

pcts = [0.99, 0.95, 0.90, 0.50, 0.10, 0.05, 0.01];
percentiles_investment =np.zeros(( len(pcts), len(Investment_frequency_months)*len(Stock_percent)))

for idx_pct, _ in enumerate(Stock_percent):
    percentiles_investment[:, idx_pct*len(Investment_frequency_months):(idx_pct+1)*len(Investment_frequency_months)] = np.quantile(investment_balance[idx_pct, :, :,1], pcts,  axis = 0)

In [ ]:
percentiles_investment.shape


(7, 105)

In [ ]:
import pandas as pd

column_index =["Monthly", "Bi-Monthly", "Quarterly", "Semi-Annually", "Annually"]
invest_methods =[]
for idx in range(100,-5, -5):
    invest_methods.append(str(100-idx)+"%sp500" + str(idx)+"%bond/")

#print("*"*100)
column_names = [ idx+ name  for name in invest_methods for idx in column_index]
#print(column_names)
## convert your array into a dataframe
df = pd.DataFrame (percentiles_investment, index = ["99th","95th", "90th", "50th", "10th", "5th", "1th"], columns = column_names)

## save to xlsx file

filepath = 'Investment percentiles.xlsx'

df.to_excel(filepath, sheet_name= "investment percentiles")

## Compare Several Investment Strategies and Get the Probabilities

In [ ]:
Stock_0percent_idx = int(0/5)
Stock_40percent_idx = int(40/5)
Stcok_60percent_idx = int(60/5)
Stcok_100percent_idx = int(100/5)

all_percents=[Stock_0percent_idx, Stock_40percent_idx, Stcok_60percent_idx, Stcok_100percent_idx]
works_sheets =["sp500_0pct", "sp500_40pct","sp500_60pct","sp500_100pct"]
for sp500_idx, sheet_name in zip (all_percents, works_sheets):

    my_balance=investment_balance[sp500_idx, :,:,1]
    num_col = my_balance.shape[1]
    comparision_matrix = np.zeros((num_col,num_col))

    for ir in range(0,num_col):
        for ic in range(0,num_col):
            comparision_matrix[ir,ic]= sum(my_balance[:, ir]>my_balance[:, ic])/my_balance.shape[0]
    with pd.ExcelWriter('invest comparions.xlsx', engine='openpyxl',
                    mode='a') as writer:
        df1 = pd.DataFrame(comparision_matrix)
        df1.to_excel(writer, sheet_name = sheet_name)

## Look at the Investment account balance after the final withdraw at T=12*(18+4)
Please take care of the index in the


dim= (len(Stock_percent), last_month_idx+1, Investment_length_months + Withdrawal_length_months, len(Investment_frequency_months), len(Withdrawal_frequency_months))
(# of port,# of rolling windows, investment and withdrawal history, investment strategies, withdraw strategies)

### Invest montthly and 100% in SP500

In [ ]:
Best_Investment_SP500_Pct = len(Stock_percent)-1 # 100% in sp500
Best_Investment_Frequency = 0 # monthly
withdrawal_balance = portfolio[Best_Investment_SP500_Pct, :, -1, Best_Investment_Frequency, :]
withdrawal_balance.shape

(1563, 5)

In [ ]:

df = pd.DataFrame (withdrawal_balance)

## save to xlsx file

filepath = 'WithdrawalPercentiles.xlsx'

df.to_excel(filepath, sheet_name= "withdrawal scenarios")

In [ ]:
num_col = withdrawal_balance.shape[1]
comparision_matrix = np.zeros((num_col,num_col))

for ir in range(0,num_col):
    for ic in range(0,num_col):
        comparision_matrix[ir,ic]= sum(withdrawal_balance[:, ir]>withdrawal_balance[:, ic])/withdrawal_balance.shape[0]

In [ ]:

filepath = 'Withdrawalcomparision_matrix.xlsx'
df = pd.DataFrame (comparision_matrix)
df.to_excel(filepath, sheet_name= "comparision_matrix")


# PPT Presentation

# Explain the 529 plan; tax benefits etc
# Explain the investment strategies; withdrawal strategies
# Explain the rolling window methods based on Robert's data
# Summarize the results at two attacched Excel workbooks
# Our contributions 1: consider 21*5*5=525 scenarios; 2: consider both investment and withdrawal strategies
# The results are consistent with the conclusion from Nobel price Laureates.

"There is **no way** to predict whether the price of stocks and bonds will go up or down over the **next
few days or weeks**. But it is **quite possible to foresee** the broad course of the prices of these assets
over **longer time periods**, such as, the next three to fve years. These fndings, which may seem both
surprising and contradictory, were made and analyzed by this year’s (2018)Laureates, **Eugene Fama, Lars
Peter Hansen and Robert Shiller**."
    
    https://www.nobelprize.org/uploads/2018/06/popular-economicsciences2013.pdf
    





In [ ]:
print("It takes--- %s seconds ---" % (time.time() - start_time))

It takes--- 1060.7247121334076 seconds ---
